# Practical Session - Motifs - Part 1

This practical session is based on what you learned during the [practical session](https://www.cs.bgu.ac.il/~tabio182/wiki.files/8-Motifs%20-%202018.pdf) regarding Motifs.

In this practical session you will:
- Find potential DnaA boxes
- Deal with actual genomes of *Vibrio cholerae*, *Thermotoga petrophila* and *E. coli*

#### How exciting! Let's start!
>Students names: Lev Gourevitch 314369547

## 1. Frequent words
### 1.1. Frequent words in a known oriC of *Vibrio cholerae*

Our plan is to begin with a bacterium in which **oriC** is known, and then determine what makes this genomic region special in order to design a computational approach for finding oriC in other bacteria. 

Our example is *Vibrio cholerae*, the pathogenic bacterium that causes cholera.

#### Solve the Frequent Words Problem.
- Input: A string **Text** and an integer **k**.
- Output: All most frequent k-mers in **Text**, and their number of occurences.

**Sample Input:**   
text = ACGTTGCATGTCGCATGATGCATGAGAGCT   
k = 4

**Sample Output:**   
\['GCAT', 'CATG'\], 3

**Running time should be: $O(|Text|*k$)**   


> **Guidance:** Use the *Counter* data structure. It is like a dictionary of items (in your case k-mers) where the key is a k-mer and the value is the number of times it appears.  
For example:


In [ ]:
from collections import Counter

words = ['cat', 'dog', 'cat']
counter = Counter()

for word in words:
    counter[word] += 1
    
print('cat', counter['cat'])

for word, count in counter.items():
    print(word, count)

In [17]:
from collections import Counter

from itertools import islice

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield "".join(result)
    for elem in it:
        result = result[1:] + (elem,)
        yield "".join(result)

def freq_words(text, k):
    result = []
    max_count = 0
    cnt = Counter()
    
    for w in window(text, k):
        cnt[w] += 1
        
    max_count = cnt.most_common(1)[0][1]
    result = [item[0] for item in cnt.items() if item[1] == max_count]
    
    return result, max_count



Test your algorithm:

In [18]:
assert freq_words('ACGTTGCATGTCGCATGATGCATGAGAGCT', 4) == (['GCAT', 'CATG'], 3)

### 1.2. oriC of Vibrio cholerae

Experiments have revealed that bacterial DnaA boxes are usually **nine** nucleotides long. 

This is the oriC of *Vibrio cholerae*. Find the most frequent 9-mers. 

In [19]:
vibrio_cholerae_oric = '''ATCAATGATCAACGTAAGCTTCTAAGCATGATCAAGGTGCTCACACAGTTTATCCACAAC
CTGAGTGGATGACATCAAGATAGGTCGTTGTATCTCCTTCCTCTCGTACTCTCATGACCA
CGGAAAGATGATCAAGAGAGGATGATTTCTTGGCCATATCGCAATGAATACTTGTGACTT
GTGCTTCCAATTGACATCTTCAGCGCCATATTGCGCTGGCCAAGGTGACGGAGCGGGATT
ACGAAAGCATGATCATGGCTGTTGTTCTGTTTATCTTGTTTTGACTGAGACTTGTTAGGA
TAGACGGTTTTTCATCACTGACTAGCCAAAGCCTTACTCTGCCTGACATCGACCGTAAAT
TGATAATGAATTTACATGCTTCCGCGACGATTTACCTCTTGATCATCGATCCGATTGAAG
ATCTTCAATTGTTAATTCTCTTGCCTCGACTCATAGCCATGATGAGCTCTTGATCATGTT
TCCTTAACCCTCTATTTTTTACGGAAGAATGATCAAGCTGCTGCTCTTGATCATCGTTTC'''

In [20]:
assert freq_words(vibrio_cholerae_oric, 9) == (['ATGATCAAG', 'CTCTTGATC', 'TCTTGATCA', 'CTTGATCAT'], 3)

### 1.3. Reverse Compliments

**ATGATCAAG** and **CTTGATCAT** are **reverse complements** of each other, resulting in the six total occurrences of these strings in *Vibrio cholerae*.

#### Find the reverse complement of a DNA string.
- Input: A DNA string Pattern.
- Output: The reverse complement of Pattern. The pattern could be lower case or upper case.

**Sample Input:**  
AAAACCCGGT

**Sample Output:**  
ACCGGGTTTT

**Sample Input:**  
aaaccc

**Sample Output:**  
gggttt

In [40]:
COMP = {'a': 't', 'c': 'g'}
COMP.update({v: k for k, v in COMP.items()})
COMP.update({k.upper(): v.upper() for k, v in COMP.items()})
def reversec(pattern):
    return "".join(reversed([COMP[c] for c in pattern]))
       

Test your algorithm:

In [41]:
reversec('AAAACCCGGT')

'ACCGGGTTTT'

### 1.4. Checking if the frequent 9-mers we found, are found in the entire genome of Vibrio cholerae

Finding a 9-mer that appears six times (either as itself or as its reverse complement) in a DNA string of length 500 is far more surprising than finding a 9-mer that appears three times (as itself). This observation leads us to the working hypothesis that **ATGATCAAG** and its reverse complement **CTTGATCAT** indeed represent DnaA boxes in Vibrio cholerae. This computational conclusion makes sense biologically because the DnaA protein that binds to DnaA boxes and initiates replication does not care which of the two strands it binds to. Thus, for our purposes, both **ATGATCAAG** and **CTTGATCAT** represent DnaA boxes.

However, before concluding that we have found the DnaA box of Vibrio cholerae, the careful bioinformatician should check if there are other short regions in the Vibrio cholerae genome exhibiting multiple occurrences of **ATGATCAAG** (or **CTTGATCAT**). After all, maybe these strings occur as repeats throughout the entire Vibrio cholerae genome, rather than just in the ori regionC. To this end, we need to solve the following problem.

#### Pattern Matching Problem: Find all occurrences of a pattern in a string.
- Input: Two strings, **Pattern** and **Genome**.
- Output: A list of integers specifying all starting positions where **Pattern** appears as a substring of **Genome**

**Sample Input:**  
ATAT  
GATATATGCATATACTT

**Sample Output:**  
1 3 9

In [42]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += 1
        
def pattern_matching(pattern, genome):
    return list(find_all(genome, pattern))


Test your algorithm:

In [43]:
pattern_matching('ATAT', 'GATATATGCATATACTT')

[1, 3, 9]

Find all starting positions (in increasing order) where **ATGATCAAG** appears as a substring in the *Vibrio cholerae* genome.

In [44]:
from pprint import pprint
vibrio_cholerae_genome = open('Vibrio_cholerae.txt').readlines()[0].strip()
mathch_indeces = pattern_matching('ATGATCAAG', vibrio_cholerae_genome)
pprint(mathch_indeces)
print(len(mathch_indeces))


[116556,
 149355,
 151913,
 152013,
 152394,
 186189,
 194276,
 200076,
 224527,
 307692,
 479770,
 610980,
 653338,
 679985,
 768828,
 878903,
 985368]
17


We discover that **ATGATCAAG** appears 17 times in *Vibrio cholerae* genome. 
Only the three occurrences of **ATGATCAAG** in oriC at starting positions 151913, 152013, and 152394  form **clumps**, i.e., appear close to each other in a small region of the genome. 

You may check that the same conclusion is reached when searching for the reverse compliment **CTTGATCAT**. We now have strong statistical evidence that **ATGATCAAG/CTTGATCAT** may represent the hidden message to DnaA to start replication.

### 1.5. Thermotoga petrophila oriC

Let's check if **ATGATCAAG/CTTGATCAT** occurs in the oriC region of *Thermotoga petrophila*, a bacterium that thrives in extremely hot environments; its name derives from its discovery in the water beneath oil reservoirs, where temperatures can exceed 80° Celsius.

In [56]:
thermotoga_petrophila_oric = """AACTCTATACCTCCTTTTTGTCGAATTTGTGTGATTTATAGAGAAAATCTTATTAACTGA
AACTAAAATGGTAGGTTTGGTGGTAGGTTTTGTGTACATTTTGTAGTATCTGATTTTTAA
TTACATACCGTATATTGTATTAAATTGACGAACAATTGCATGGAATTGAATATATGCAAA
ACAAACCTACCACCAAACTCTGTATTGACCATTTTAGGACAACTTCAGGGTGGTAGGTTT
CTGAAGCTCTCATCAATAGACTATTTTAGTCTTTACAAACAATATTACCGTTCAGATTCA
AGATTCTACAACGCTGTTTTAATGGGCGTTGCAGAAAACTTACCACCTAAAATCCAGTAT
CCAAGCCGATTTCAGAGAAACCTACCACTTACCTACCACTTACCTACCACCCGGGTGGTA
AGTTGCAGACATTATTAAAAACCTCATCAGAAGCTTGTTCAAAAATTTCAATACTCGAAA
CCTACCACCTGCGTCCCCTATTATTTACTACTACTAATAATAGCAGTATAATTGATCTGA"""

thermotoga_petrophila_oric = "".join(thermotoga_petrophila_oric.split('\n'))

print(pattern_matching('ATGATCAAG', thermotoga_petrophila_oric))
print(pattern_matching('CTTGATCAT', thermotoga_petrophila_oric))

[]
[]


In [57]:
assert pattern_matching('ATGATCAAG', thermotoga_petrophila_oric) == []
assert pattern_matching('CTTGATCAT', thermotoga_petrophila_oric) == []

This region does not contain a single occurrence of **ATGATCAAG** or **CTTGATCAT**! Thus, different bacteria may use different DnaA boxes as "hidden messages" to the DnaA protein.

### 1.6. Frequent words in *Thermotoga petrophila* oriC
We want to find **9-mers** in oriC of *Thermotoga petrophila* appearing **5** times or more (including their reverse compliment).
Let's modify *freq_words* function to achieve that.

#### Modified Frequent Words Problem.
- Input: A string **Text** and integers **k, c**.
- Output: Pairs of k-mers and their reverse compliments in **Text** that apear **c** or more times, and their number of occurrences, without duplications.

**Sample Input:**   
text = ACGTTGCATTCGCATATGATTCATGAGAGCT   
k = 4  
c = 3

**Sample Output:**   
\[('ATGA', 2, 'TCAT', 1, 3)\]

In this example k-mer **ATGA** appers 2 times, its reverse compliment **TCAT** appears 1 time, total occurrences 3 times.

In [58]:
from collections import Counter

def freq_words2(text, k, c):
    cnt = Counter()
    pairs = []
    
    for w in window(text, k):
        cnt[w] += 1
        
    unique_pairs = {tuple(sorted([kmer, reversec(kmer)])) for kmer in cnt}
    
    for kmer, rev_kmer in unique_pairs:
        if cnt[kmer] + cnt[rev_kmer] >= c:
            pairs.append((kmer, cnt[kmer], rev_kmer, cnt[rev_kmer], cnt[kmer] + cnt[rev_kmer]))
        
    return sorted(pairs, key=lambda t: t[4], reverse=True)
    
    

Test your algorithm:

In [59]:
freqs = freq_words2('ACGTTGCATTCGCATATGATTCATGAGAGCT', 4, 3)
print(freqs)
assert freqs == [('ATGA', 2, 'TCAT', 1, 3)]

[('ATGA', 2, 'TCAT', 1, 3)]


And now in the oriC of *Thermotoga petrophila*.

In [60]:
freq_words2(thermotoga_petrophila_oric, 9, 5)

[('ACCTACCAC', 5, 'GTGGTAGGT', 2, 7),
 ('AACCTACCA', 3, 'TGGTAGGTT', 3, 6),
 ('AAACCTACC', 3, 'GGTAGGTTT', 3, 6),
 ('CCTACCACC', 3, 'GGTGGTAGG', 2, 5)]

It is **extremely unlikely** that 4 different 9-mers will occur so frequently within the same short region in a random string. We will cheat a little and consult with *Ori-Finder*, a software tool for finding replication origins in DNA sequences. 

This software chooses **CCTACCACC** (along with its reverse complement **GGTGGTAGG**) as a working hypothesis for the DnaA box in Thermotoga petrophila. Together, these two complementary 9-mers appear five times in the replication origin.